# OpenStreetMap Project
# Data Wrangling with MongoDB
### Tim Moore

Map Area: Seattle, WA, United States

https://mapzen.com/data/metro-extracts/#seattle-washington

http://www.openstreetmap.org/relation/237385

## 1. Problems Encountered in the Map

The Seattle .xml file downloaded from mapzen.com turned out to be quite large at 1.46 GB once it was unzipped. I utilized the provided code from the project details page to work on 1/10th of it initially. Once I had my wrangling helper functions, file processing function and database upload working correctly on a smaller scale, I began running my code using the full dataset. 

A few of the larger challenges I had were:
- Standardizing postal codes (the Seattle map has some overlap with the Victoria B.C. Map, and Canadian postal codes are in a different format)
- Cleaning up and standardizing city names (capitalizing consistently, removing state info if provided, handling a few one-off cases)
- Cleaning up and standardizing street names


### Standardizing postal codes

My map data included parts of Canada in the Seattle data, this caused issues mostly with post codes. Canadian post codes are in a format of A1A A1A though many users combined the two groups together. This allowed me to become more of an expert with the re module of python as I built the function to standardize all postcodes before they were written to the JSON file that was passed into MongoDB. 

I decided to update all Canadian post codes into XXX XXX format and truncated US post codes down to only 5 digits. I decided to return the None type for any input that wasn't correct but I was unable to correct it due to included data. 


### Cleaning up and standardizing City Names

I found some city names that included other identifying features, for example 'Oak Harbor (whidbey island)'. I decided that additional levels of identification were ok here and changed this example into 'Oak Harbor Whidbey Island'. I also removed any state/province information that may have been included by the creator, and made sure every city name was capitalized.


### Cleaning up and standardizing Street Names

Most of the street names that needed to be cleaned up turned out to be Directional street names (NE 116th st becomes Northeast 116th Street). I updated the strings prior to saving as a JSON file to limit the number of times I had to open and close files due to the size of this map.

# Helper functions and Global Variables

In [1]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
import codecs
import json
import sys
sys.path.append('c:/python27/lib/site-packages')

import pymongo
from pymongo import MongoClient

OSM_FILE = "c:/users/moore/documents/udacity/project 3/seattle_washington.osm/seattle_washington.osm"


street_type_re = re.compile(r'\b\S+?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", 'Northeast', 'Northwest', 'Southwest', 'Southeast', 'East',
           'West', 'South', 'North', 'Terrace']

mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Dr": "Drive",
            "Dr.": "Drive",
            "Ct": "Court",
            "Ct.": "Court",
            "Pl": "Place",
            "Pl.": "Place",
            "Sq": "Square",
            "Sq.": "Square",
            "Ln": "Lane",
            "Ln.": "Lane",
            "Rd.": "Road",
            "Rd" : "Road",
            "Tr" : "Trail",
            "Tr.": "Trail",
            "Pkwy": "Parkway",
            "Pkwy.": "Parkway",
            "NE": "Northeast",
           'NW': 'Northwest',
           'SW': 'Southwest',
           'SE': 'Southeast',
           'E': 'East',
           'W': 'West',
           'S': 'South',
           'N': 'North',
           'Ter': 'Terrace'           
            }
## Helper functions

def aggregate(db, pipeline):
    return [doc for doc in seattle.aggregate(pipeline)]

def update_name(name, mapping):
    m = street_type_re.search(name)
    other_street_types = []
    if m:
        street_type = m.group()
        if street_type in mapping.keys():
            name = re.sub(street_type, mapping[street_type],name)
        else:
            other_street_types.append(street_type)
    return name

def update_postcode(postcode):
    if re.search(r'\d{5}', postcode):
        postcode = re.search(r'\d{5}', postcode).group()
    elif re.search(r'\d{5}(?:[-\s]\d{4})?$', postcode):
        postcode = re.search( r'^\d{5}',postcode).group()
    elif re.search(r'\w{6}', postcode):
        firstpart, secondpart = postcode[:len(postcode)/2], postcode[len(postcode)/2:]
        postcode = firstpart + ' ' + secondpart
    elif re.search(r'(\w{3})\s(\w{3})', postcode):
        postcode = re.search(r'\w{3}\s(\w{3})', postcode).group()
    elif re.search(r'[WA]', postcode):
        postcode = None
    else:
        return postcode
    return postcode   

def update_city(city):
    try:
        correct_name = city.split(',')
        if len(city.split(';')) == 2:
            correct_name = city.split(';')[0]
        name = ''
        for subname in correct_name[0].split():
            name += subname.strip('()').capitalize() + ' '
        return name.rstrip(' ')
    except:
        return city.capitalize()

# Processing .xml file and shaping it into the desired JSON structure

In [2]:
## Processing each element from the .xml dataset and manipulating it into our desired JSON shape

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

GENERAL = [ 'visible', 'amenity', 'cuisine', 'name', 'phone']

def shape_element(element):
    node = {}
    node['created'] = {}
    address = {}
    node['pos'] = []
    if element.tag == "node" or element.tag == "way" :
        for x in CREATED:
            if x in element.attrib:
                node['created'][x] = element.attrib[x]
        node['id'] = element.attrib['id']
        for x in GENERAL:
            if x in element.attrib:
                node[x] = element.attrib[x]
        node['type'] = element.tag
        try:
            node['pos'].append(float(element.attrib['lat']))
            node['pos'].append(float(element.attrib['lon']))
        except: 
            pass
        node_refs = []
        for subtag in element:
            if subtag.tag == 'tag':
                if re.search(problemchars, subtag.get('k')):
                    pass
                elif re.search(r'\w+:\w+:\w+', subtag.get('k')):
                    pass
                elif subtag.get('k').startswith('addr:'):
                    if re.search(r'addr:street', subtag.get('k')):
                        address["street"]=update_name(subtag.attrib['v'],mapping)             
                    
                    elif re.findall('post',subtag.get('k')):
                        address["postcode"]=update_postcode(subtag.attrib["v"])
                        # This takes care of a single issue with user input
                        if address['postcode'] == 'W Lake Samma mish Pkwy NE':
                            address['postcode'] = None
                            address['street'] = 'W Lake Sammamish Pkwy NE'
                    elif re.findall('city',subtag.get('k')):
                        address["city"]=update_city(subtag.attrib['v'])
                    else:
                        address[subtag.get('k')[5:]] = subtag.get('v')
                else:
                    node[subtag.get('k')] = subtag.get('v')
            else:
                if subtag.tag == 'nd':
                    node_refs.append(subtag.get('ref'))        
                else:
                    pass
        if node_refs:
            node['node_refs'] = node_refs
        if address:
            node['address'] = address
        
        return node
    else:
        return None
    
def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")

    return data

# Processing .xml file and inserting it to MongoDB

In [4]:
data = process_map(OSM_FILE, False)
client = MongoClient('mongodb://localhost:27017')
client.drop_database('seattle')
db = client.seattle
seattle = db.seattle
result = seattle.insert_many(data)

# Data Overview
This section contains basic statisticcs about the dataset I investigated as well as the MongoDB queries used to gather them.

### File size
seattle_washington.osm     ...........       1.46 GB

seattle_washington.osm.json   ...    1.66 GB

### Total number of documents

In [5]:
seattle.find().count()

7711454

### Total number of nodes

In [6]:
seattle.find({'type': 'node'}).count()

7034903

### Total number of ways

In [7]:
seattle.find({'type': 'way'}).count()

676450

### Top contributing user

In [8]:
pipeline = [{'$group': {'_id': '$created.user', 'count' : {'$sum': 1}}},
            {'$sort': {'count':-1}}, 
            {"$limit":1}]
top_user = aggregate(seattle, pipeline)
pprint.pprint(top_user)

[{u'_id': u'Glassman', u'count': 1237556}]


### Top 10 contributors

In [9]:
pipeline = [{'$group': {'_id': '$created.user', 'count' : {'$sum': 1}}},
            {'$sort': {'count':-1}}, 
            {"$limit":10}]
top_ten_users = aggregate(seattle, pipeline)
pprint.pprint(top_ten_users)

[{u'_id': u'Glassman', u'count': 1237556},
 {u'_id': u'SeattleImport', u'count': 745875},
 {u'_id': u'tylerritchie', u'count': 661401},
 {u'_id': u'woodpeck_fixbot', u'count': 615971},
 {u'_id': u'alester', u'count': 329568},
 {u'_id': u'Glassman_Import', u'count': 240624},
 {u'_id': u'STBrenden', u'count': 223596},
 {u'_id': u'Brad Meteor', u'count': 179996},
 {u'_id': u'Amoebabadass', u'count': 168030},
 {u'_id': u'Omnific', u'count': 124055}]


### Number of users with only a single post

In [10]:
pipeline = [{'$group': {'_id': '$created.user', 'count': {'$sum': 1}}},
           {'$group': {'_id': '$count', 'num_users': {'$sum':1}}},
           {'$sort':{'_id':1}}, 
           {'$limit':1}]
single_poster_count = aggregate(seattle, pipeline)
pprint.pprint(single_poster_count)

[{u'_id': 1, u'num_users': 531}]


### Postal codes showing corrected US and Canadian formats

In [18]:
def make_pipeline():
    pipeline = [
        {"$match": {"address.postcode":{"$exists":1}}}, 
        {"$group": {"_id": "$address.postcode", "count": {"$sum":1}}},
        {"$sort": {"count": 1}}
    ]
    return pipeline

pipeline = make_pipeline()
post_code_results = aggregate(seattle, pipeline)
pprint.pprint(post_code_results[:10])

[{u'_id': u'V8R 1S1', u'count': 1},
 {u'_id': u'V8W 2G5', u'count': 1},
 {u'_id': u'V8W 2A4', u'count': 1},
 {u'_id': u'V8T 4Y3', u'count': 1},
 {u'_id': u'V8T 5J8', u'count': 1},
 {u'_id': u'V9Z 0A1', u'count': 1},
 {u'_id': u'98253', u'count': 1},
 {u'_id': u'V8W 1S7', u'count': 1},
 {u'_id': u'98191', u'count': 1},
 {u'_id': u'4139', u'count': 1}]


### I was confused by the city named "Capital H Part 1" 
I first assumed this was due to users incorrectly assigning places to the Seattle Neighborhood Capitol Hill. After googling around, I realized this is a subsection of the Gulf Islands (near Victoria, B.C.) so I standardized the capitalization. 

This investigation as well as the high number of Salish names led me to decide not to spell-check city names. Instead this was the singular area I relied upon user input to be correct.

In [22]:
def make_pipeline():
    pipeline = [
        
        {"$match": {"address.city":{"$exists":1}}}, 
        {"$group": {"_id": "$address.city", "count": {"$sum":1}}},
        {"$sort": {"count": -1}}
    ]
    return pipeline

pipeline = make_pipeline()
city_results = aggregate(db, pipeline)
pprint.pprint(city_results[12])

{u'_id': u'Capital H Part 1', u'count': 739}


### Top 20 amenities
I'm happy to see the greater Seattle metro area's unique personality is included within this map. We can see users have identified 1/3 the number of bike parking locations as there are car parking locations. As well as more cafe's than bars and pubs combined (many better than Starbucks even)

In [23]:
def make_pipeline():
    pipeline = [
        
        {"$match": {"amenity":{"$exists":1}}}, 
        {"$group": {"_id": "$amenity", "count": {"$sum":1}}},
        {"$sort": {"count": -1}},
        {"$limit": 20}
    ]
    return pipeline

pipeline = make_pipeline()
amenities = aggregate(db, pipeline)
pprint.pprint(amenities)

[{u'_id': u'parking', u'count': 9014},
 {u'_id': u'bicycle_parking', u'count': 3083},
 {u'_id': u'restaurant', u'count': 2838},
 {u'_id': u'school', u'count': 2614},
 {u'_id': u'bench', u'count': 1983},
 {u'_id': u'place_of_worship', u'count': 1561},
 {u'_id': u'fast_food', u'count': 1352},
 {u'_id': u'cafe', u'count': 1210},
 {u'_id': u'fuel', u'count': 1063},
 {u'_id': u'toilets', u'count': 837},
 {u'_id': u'bank', u'count': 814},
 {u'_id': u'waste_basket', u'count': 791},
 {u'_id': u'bar', u'count': 307},
 {u'_id': u'pub', u'count': 292},
 {u'_id': u'post_box', u'count': 290},
 {u'_id': u'fire_station', u'count': 282},
 {u'_id': u'drinking_water', u'count': 275},
 {u'_id': u'pharmacy', u'count': 274},
 {u'_id': u'shelter', u'count': 249},
 {u'_id': u'parking_entrance', u'count': 242}]


# Additional Ideas

## Incentivizing a larger user/creator network
Within the Seattle dataset the top contributors are clearly creating much more data than those outside the top 10 contributers. For example the top contributor has more than 10x the number of contributions as the tenth highest contributor. This disparitiy in the number of contributions each user has submitted, will disincentivize some from becoming new users.

OSM could attempt to motivate more contributors by focusing upon sub-areas that are interesting for users. For example if OSM were to put out a challenge for the highest number of contributions improving a specific JSON field/sub-document (any of the amenity sub-categories - for example post your favorite coffee shop). By rotating the focus of these improvement pushes, updating the challenges often and broadcasting these challenges to a wide audience, OSM may be able to bring in more contributions from their less frequent users. I believe if OSM were to institute games of this variety, it may be key to building a more involved user base resulting in improved OSM data, especially if the newest challenge represents a priority for said user. 


## Improving the Amenities sub-document
I feel the amenities field is overly broad without providing a lot of help to users. There are lots of top amenities listed above that likely have more than one amenity that users would be interested in knowing about. If you travel via bicycle it is helpful to know bike parking exists at/near your destination.

When I am traveling to a foreign city, a consistent challenge is finding a public restroom while I'm out and exploring all day. Many cafe's, pubs and bars likely have a public restroom as well. If the Amenities field was changed into a sub-document similarly to how the address field was handled, OSM would be able to provide more in-depth applicable information for end-users. Providing more amenity data would enable more relevant querying options, improving end-user experience. By increasing the utility each end-user of OSM gains, OSM will be able to grow the network of their user/creator base. 

# Conclusion
The Seattle data is overly large covering nearly half the State of Washington and a bit of Canada for good measure. Covering international data brought additional challenges for data cleaning, but these were great practice going forward as Data Analysts can never assume the data is correct. 

The size of the OSM file provided an additional challenge, occasionally I would wait 10-20 minutes for my JSON file to be built. It was fun to see my computer using all 16 GB of RAM while crunching the data though. After dealing with such a large file I understand the importance of running small tests of my data for debugging/code performance. I was happy to not have any issues running the whole dataset once my code was functional. 